# Script to create graph and reactions from input matrix


In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import os 
import numpy as np
import os
import pandas as pd
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from tqdm_joblib import tqdm_joblib

In [3]:
state_vars_per_gene = ["is_bursting", "unspliced_mRNA", "spliced_mRNA", "protein",
                      "k_on_adjusted", "total_mRNA", 
                      "mRNA_ever_produced", "protein_ever_produced"]
#Define indices for each state var
state_var_indices = {var: i for i, var in enumerate(state_vars_per_gene)}

In [4]:
#To read the input matrix
def read_input_matrix(path_to_matrix):
    """
    Reads the input matrix from the specified file path and counts number of genes.
    
    Parameters:
    path_to_matrix (str): The file path to the input matrix.
    
    Returns:
    np.ndarray: The input matrix as a NumPy array.
    """
    matrix = np.loadtxt(path_to_matrix, dtype='i', delimiter=',')
    if matrix.ndim == 0:
        matrix = np.array([[matrix]])

    # print(type(matrix))
    # print(matrix.shape)
    return matrix.shape[0], matrix

In [5]:
#Read the parameters
def assign_parameters_to_genes(csv_path, rows=None, n_random=3):
    """
    Select specific rows or random rows from parameter CSV and map to genes.
    
    Args:
        csv_path (str): Path to CSV file
        rows (list, optional): Specific row indices to select. If None, selects random rows
        n_random (int): Number of random rows if rows=None
    
    Returns:
        tuple: (param_dict, row_mapping)
            param_dict: {gene_id: {param_name: value}}
            row_mapping: {gene_id: row_number}
    """
    df = pd.read_csv(csv_path, index_col=0)

    # If no specific rows are provided, select random rows to assugn to genes
    if rows is None:
        rows = np.random.choice(df.index, size=min(n_random, len(df)), replace=True)
    
    param_dict = {}
    row_mapping = {}
    
    for i, row in enumerate(rows):
        if row not in df.index:
            raise ValueError(f"Row {row} not found in the DataFrame.")
        gene_id = f"gene_{i+1}"
        param_dict[gene_id] = df.loc[row].to_dict()
        row_mapping[gene_id] = row
    # Create a matrix where genes are rows and parameters are columns
    param_matrix = pd.DataFrame.from_dict(param_dict, orient='index')

    #Add additional calculations for degradation, transcription rate, splicing rate, etc.
    param_matrix['mRNA_degradation_rate'] = np.log(2)/param_matrix['mrna_half_life']
    param_matrix['protein_degradation_rate'] = np.log(2)/param_matrix['protein_half_life']
    param_matrix['transcription_rate'] = param_matrix['burst_size']*param_matrix['k_off']

    #Remove old columns that are not needed
    param_matrix.drop(columns=['mrna_half_life', 'protein_half_life', 'burst_size'], inplace=True, errors='ignore')

    return param_matrix, row_mapping

In [6]:
def save_complete_snapshot(state_array, state_var_indices, timestep=None):
    """
    Reshape from (n_genes, n_state_vars, n_cells) to (n_cells, n_genes × n_state_vars) DataFrame
    """
    n_genes, n_state_vars, n_cells = state_array.shape

    # Prepare data columns
    columns = []
    data = []

    for gene_idx in range(n_genes):
        for var_name, var_idx in state_var_indices.items():
            col_name = f"gene_{gene_idx+1}_{var_name}"
            columns.append(col_name)
            data.append(state_array[gene_idx, var_idx, :])  # shape (n_cells,)

    # Stack and transpose to (n_cells, n_features)
    df_data = np.stack(data, axis=1)  # shape: (n_cells, n_genes × n_state_vars)
    df = pd.DataFrame(df_data, columns=columns)
    df["cell_id"] = df.index # Add cell IDs as a column
    # Add metadata
    if timestep is not None:
        df["timestep"] = timestep
    return df

In [7]:
# from scipy.optimize import fsolve
# import numpy as np

# def get_steady_state_protein_levels(param_matrix, interaction_matrix, global_params=None, target_hill=0.5):
#     """
#     Calculate steady-state protein levels for all genes, to use as EC50 values.

#     Args:
#         param_matrix (pd.DataFrame): Parameters for each gene.
#         interaction_matrix (np.ndarray): shape (n_genes, n_genes)
#         global_params (dict): Global constants.
#         target_hill (float): Desired hill response at steady state (default 0.5)

#     Returns:
#         np.ndarray: steady-state protein levels for each gene
#     """

#     if global_params is None:
#         global_params = {
#             "splicing_rate": np.log(2)/(7/60),
#             "max_effect": 16,
#             "n": 2
#         }

#     splicing_rate = global_params["splicing_rate"]
#     max_effect = global_params["max_effect"]
#     n = global_params["n"]

#     n_genes = param_matrix.shape[0]
#     k_on = param_matrix["k_on"].values
#     k_off = param_matrix["k_off"].values
#     txn_rate = param_matrix["transcription_rate"].values
#     m_deg = param_matrix["mRNA_degradation_rate"].values
#     p_prod = param_matrix["protein_production_rate"].values
#     p_deg = param_matrix["protein_degradation_rate"].values

#     def system(P):
#         eqs = []
#         for i in range(n_genes):
#             regulators = np.where(interaction_matrix[:, i] != 0)[0]
#             if len(regulators) > 0:
#                 # For self-consistent EC50: assume Hill(P[i]) = 0.5 at steady state
#                 hill_sum = np.sum(target_hill * interaction_matrix[regulators, i]) 
#                 print(f"For gene{i}, hill_sum = {hill_sum}")               
#             else:
#                 hill_sum = 0
#             # if len(regulators) > 0:
#             #     # Compute weighted log-hill terms
#             #     hill_log = np.log(np.clip(target_hill, 1e-10, None))  # prevent log(0)
#             #     weights = interaction_matrix[regulators, i]
#             #     log_weighted = hill_log * weights

#             #     # Compute product of hill_j ** weight_j
#             #     hill_prod = np.exp(np.sum(log_weighted))

#             #     # Clip final product (not individual terms)
#             #     hill_prod_clipped = np.clip(hill_prod, 1e-10, 1)
#             # else:
#             #     hill_prod_clipped = 0  # No regulators → no regulatory input

#             # # Final adjusted k_on
#             # k_on_eff = k_on[i] + max_effect * hill_prod_clipped

#             k_on_eff = k_on[i] + max_effect * hill_sum
#             k_on_eff = np.clip(k_on_eff, 1e-10, None)  # Ensure non-negative
#             # print(f"Gene {i+1}: k_on_eff = {k_on_eff}, Hill sum = {hill_sum}")
#             # Bursting + splicing + translation
#             burst_prob = k_on_eff / (k_on_eff + k_off[i])
#             print(f"burst_prob: {burst_prob}")
#             unspliced = txn_rate[i] * burst_prob / (m_deg[i] + splicing_rate)
#             spliced = unspliced * splicing_rate / m_deg[i]
#             protein = spliced * p_prod[i] / p_deg[i]
#             print(f"Gene {i+1}: k_on_eff = {k_on_eff}, Hill sum = {hill_sum}")
#             # Bursting + splicing + translation
#             print(f"splicing rate = {splicing_rate}, m_deg = {m_deg[i]}")

#             print(f"Unspliced Level = {unspliced}")
#             print(f"Spliced Level = {spliced}")
#             print(f"Spliced = {spliced}, p_prod[i] = {p_prod[i]} and p_deg = {p_deg[i]}")

#             print(f"Protein = {protein}")
#             eqs.append(protein - P[i])
#         return eqs

#     # Initial guess: unregulated steady state for each gene
#     guess = []
#     for i in range(n_genes):
#         b = k_on[i] / (k_on[i] + k_off[i])
#         u = txn_rate[i] * b / (m_deg[i] + splicing_rate)
#         s = u * splicing_rate / m_deg[i]
#         p = s * p_prod[i] / p_deg[i]
#         guess.append(p)

#     try:
#         result = fsolve(system, guess)
#         return np.maximum(result, 0.1)  # avoid zero
#     except Exception as e:
#         print(f"⚠️ Failed to solve steady state: {e}")
#         return np.array(guess)


In [8]:
import numpy as np
from scipy.optimize import fsolve

def get_steady_state_protein_levels(param_matrix, interaction_matrix, 
                                  global_params=None, target_hill=0.5, verbose=False):
    """
    Calculate steady-state protein levels for all genes using deterministic equations.

    Args:
        param_matrix (pd.DataFrame): Parameters for each gene with columns:
            ['k_on', 'k_off', 'transcription_rate', 'mRNA_degradation_rate', 
             'protein_production_rate', 'protein_degradation_rate']
        interaction_matrix (np.ndarray): Regulatory interactions, shape (n_genes, n_genes)
            interaction_matrix[i, j] = effect of gene i on gene j
        global_params (dict, optional): Global constants
        target_hill (float): Target Hill response at steady state (default 0.5)
        verbose (bool): Print debugging information

    Returns:
        np.ndarray: Steady-state protein levels for each gene
    """
    
    # Set default global parameters
    if global_params is None:
        global_params = {
            "splicing_rate": np.log(2) / (7/60),  # 7-minute mRNA maturation
            "max_effect": 16,                      # Maximum regulatory effect
            "n": 2                                 # Hill coefficient
        }

    max_effect = global_params["max_effect"]
    n_genes = param_matrix.shape[0]
    
    # Calculate k_on_eff for each gene based on regulatory inputs
    k_on_original = param_matrix["k_on"].values
    k_on_eff = k_on_original.copy()
    
    for i in range(n_genes):
        # Find regulatory inputs
        regulators = np.where(interaction_matrix[:, i] != 0)[0]
        
        if len(regulators) > 0:
            # Calculate regulatory effect (assuming target_hill response)
            hill_sum = np.sum(target_hill * interaction_matrix[regulators, i])
            regulatory_effect = max_effect * hill_sum
            k_on_eff[i] = k_on_original[i] + regulatory_effect
            
            if verbose:
                print(f"Gene {i}: k_on={k_on_original[i]:.3f} -> k_on_eff={k_on_eff[i]:.3f} "
                      f"(regulatory effect: {regulatory_effect:.3f})")
    
    # Create modified parameter matrix with effective k_on values
    param_matrix_eff = param_matrix.copy()
    param_matrix_eff["k_on"] = np.maximum(k_on_eff, 1e-10)  # Ensure positive
    
    # Calculate protein levels using the unregulated function with effective k_on
    protein_levels = calculate_unregulated_protein_levels(param_matrix_eff, global_params)
    
    if verbose:
        print(f"Steady-state protein levels: {protein_levels}")
    
    return np.maximum(protein_levels, 0.1)  # Ensure positive protein levels


def calculate_unregulated_protein_levels(param_matrix, global_params=None):
    """
    Calculate protein levels without any regulation (for comparison/initialization).
    
    Args:
        param_matrix (pd.DataFrame): Gene parameters
        global_params (dict, optional): Global constants
        
    Returns:
        np.ndarray: Unregulated steady-state protein levels
    """
    if global_params is None:
        global_params = {"splicing_rate": np.log(2) / (7/60)}
    
    splicing_rate = global_params["splicing_rate"]
    
    # Extract parameters
    k_on = param_matrix["k_on"].values
    k_off = param_matrix["k_off"].values
    txn_rate = param_matrix["transcription_rate"].values
    m_deg = param_matrix["mRNA_degradation_rate"].values
    p_prod = param_matrix["protein_production_rate"].values
    p_deg = param_matrix["protein_degradation_rate"].values
    
    # Calculate unregulated levels
    burst_prob = k_on / (k_on + k_off)
    unspliced_mRNA = txn_rate * burst_prob / (m_deg + splicing_rate)
    spliced_mRNA = unspliced_mRNA * splicing_rate / m_deg
    protein_levels = spliced_mRNA * p_prod / p_deg
    
    return protein_levels

In [9]:
# def get_steady_state_protein_levels(param_matrix, interaction_matrix, global_params=None, tol=1e-4, max_iter=100):
#     """
#     Solve for steady-state protein levels where each gene's EC50 is set to its own 
#     steady-state protein level (i.e., K_i = P_i). Uses fixed-point iteration.

#     Args:
#         param_matrix (pd.DataFrame): Parameters for each gene.
#         interaction_matrix (np.ndarray): shape (n_genes, n_genes), 
#                                          +ve for activation, -ve for repression.
#         global_params (dict): Dictionary with keys 'splicing_rate', 'max_effect', 'n'
#         tol (float): Tolerance for convergence on EC50 self-consistency
#         max_iter (int): Max number of fixed-point iterations

#     Returns:
#         np.ndarray: self-consistent steady-state protein levels
#     """

#     if global_params is None:
#         global_params = {
#             "splicing_rate": np.log(2)/(7/60),
#             "max_effect": 16,
#             "n": 2,
#         }

#     splicing_rate = global_params["splicing_rate"]
#     max_effect = global_params["max_effect"]
#     n = global_params["n"]

#     n_genes = param_matrix.shape[0]
#     k_on = param_matrix["k_on"].values
#     k_off = param_matrix["k_off"].values
#     txn_rate = param_matrix["transcription_rate"].values
#     m_deg = param_matrix["mRNA_degradation_rate"].values
#     p_prod = param_matrix["protein_production_rate"].values
#     p_deg = param_matrix["protein_degradation_rate"].values

#     # Initial guess: unregulated steady state
#     K = []
#     for i in range(n_genes):
#         b = k_on[i] / (k_on[i] + k_off[i])
#         u = txn_rate[i] * b / (m_deg[i] + splicing_rate)
#         s = u * splicing_rate / m_deg[i]
#         p = s * p_prod[i] / p_deg[i]
#         K.append(p)
#     K = np.array(K)

#     def system(P, K_fixed):
#         eqs = []
#         for i in range(n_genes):
#             regulators = np.where(interaction_matrix[:, i] != 0)[0]
#             hill_sum = 0.0
#             for j in regulators:
#                 weight = interaction_matrix[j, i]
#                 P_j = max(P[j], 1e-8)
#                 K_i = max(K_fixed[i], 1e-8)
#                 hill = P_j**n / (K_i**n + P_j**n)
#                 hill_sum += weight * hill

#             k_on_eff = k_on[i] + max_effect * hill_sum
#             k_on_eff = max(k_on_eff, 1e-8)

#             burst_prob = k_on_eff / (k_on_eff + k_off[i])
#             unspliced = txn_rate[i] * burst_prob / (m_deg[i] + splicing_rate)
#             spliced = unspliced * splicing_rate / m_deg[i]
#             protein = spliced * p_prod[i] / p_deg[i]

#             eqs.append(protein - P[i])
#         return eqs

#     # Fixed-point iteration
#     for iteration in range(max_iter):
#         P = fsolve(lambda P: system(P, K), K)
#         delta = np.linalg.norm(P - K)
#         print(f"[Iteration {iteration}] Δ = {delta:.6f}")
#         if delta < tol:
#             break
#         K = P.copy()

#     return np.maximum(P, 0.1)


In [10]:
def get_max_protein_level(param_matrix, interaction_matrix, global_params=None, target_hill=0.5, threshold_frac = 0.95):
    """
    Calculate steady-state protein levels for all genes, to use as EC50 values.

    Args:
        param_matrix (pd.DataFrame): Parameters for each gene.
        interaction_matrix (np.ndarray): shape (n_genes, n_genes)
        global_params (dict): Global constants.
        target_hill (float): Desired hill response at steady state (default 0.5)

    Returns:
        np.ndarray: steady-state protein levels for each gene
    """

    if global_params is None:
        global_params = {
            "splicing_rate": np.log(2)/(7/60),
            "max_effect": 16,
            "n": 2
        }

    splicing_rate = global_params["splicing_rate"]
    max_effect = global_params["max_effect"]
    n = global_params["n"]

    n_genes = param_matrix.shape[0]
    k_on = param_matrix["k_on"].values
    k_off = param_matrix["k_off"].values
    txn_rate = param_matrix["transcription_rate"].values
    m_deg = param_matrix["mRNA_degradation_rate"].values
    p_prod = param_matrix["protein_production_rate"].values
    p_deg = param_matrix["protein_degradation_rate"].values

    def system(P):
        eqs = []
        for i in range(n_genes):
            regulators = np.where(interaction_matrix[:, i] != 0)[0]
            if len(regulators) > 0:
                # For self-consistent EC50: assume Hill(P[i]) = 0.5 at steady state
                hill_sum = np.sum(target_hill * interaction_matrix[regulators, i])                
            else:
                hill_sum = 0
            # if len(regulators) > 0:
            #     # Compute weighted log-hill terms
            #     hill_log = np.log(np.clip(target_hill, 1e-10, None))  # prevent log(0)
            #     weights = interaction_matrix[regulators, i]
            #     log_weighted = hill_log * weights

            #     # Compute product of hill_j ** weight_j
            #     hill_prod = np.exp(np.sum(log_weighted))

            #     # Clip final product (not individual terms)
            #     hill_prod_clipped = np.clip(hill_prod, 1e-10, 1)
            # else:
            #     hill_prod_clipped = 0  # No regulators → no regulatory input

            # # Final adjusted k_on
            # k_on_eff = k_on[i] + max_effect * hill_prod_clipped

            k_on_eff = k_on[i] + max_effect * hill_sum
            k_on_eff = np.clip(k_on_eff, 1e-10, None)  # Ensure non-negative
            # print(f"Gene {i+1}: k_on_eff = {k_on_eff}, Hill sum = {hill_sum}")
            # Bursting + splicing + translation
            unspliced = txn_rate[i] *threshold_frac / (m_deg[i] + splicing_rate)
            # print(f"Unspliced Level = {unspliced}")
            spliced = unspliced * splicing_rate / m_deg[i]
            # print(f"splicing rate = {splicing_rate}, m_deg = {m_deg[i]}")
            # print(f"Spliced Level = {spliced}")
            protein = spliced * p_prod[i] / p_deg[i]
            # print(f"Spliced = {spliced}, p_prod[i] = {p_prod[i]} and p_deg = {p_deg[i]}")
            # print(f"Protein = {protein}")

            eqs.append(protein - P[i])
        return eqs

    # Initial guess: unregulated steady state for each gene
    guess = []
    for i in range(n_genes):
        b = k_on[i] / (k_on[i] + k_off[i])
        u = txn_rate[i] / (m_deg[i] + splicing_rate)
        s = u * splicing_rate / m_deg[i]
        p = s * p_prod[i] / p_deg[i]
        guess.append(p)

    try:
        result = fsolve(system, guess)
        return np.maximum(result, 0.1)  # avoid zero
    except Exception as e:
        print(f"⚠️ Failed to solve steady state: {e}")
        return np.array(guess)

In [11]:
def create_EC50_vector(param_matrix, interaction_matrix, global_params=None, threshold_frac = None):
    """
    Create EC50 matrix where EC50[i,j] = steady-state protein level of regulator j
    when it regulates target i.
    
    Args:
        param_matrix: DataFrame with parameter values
        interaction_matrix: regulatory interaction matrix
        global_params: global parameters
    
    Returns:
        np.ndarray: EC50 matrix (shape: n_genes, n_genes)
    """
    # Get steady-state protein levels
    steady_state_protein = get_steady_state_protein_levels(param_matrix, interaction_matrix, global_params)
    # print(f"In create_EC50_vector, threshold fraction = {threshold_frac}")
    if  threshold_frac == None:
        average_bursting_only = get_steady_state_protein_levels(param_matrix, interaction_matrix, global_params)
    else:
        average_bursting_only = get_max_protein_level(param_matrix, interaction_matrix, global_params, threshold_frac = threshold_frac)
    # return np.maximum(steady_state_protein, 1e-12)  # Avoid 0 in denominator
    return np.maximum(steady_state_protein, 1e-12), np.maximum(average_bursting_only, 1e-12)  # Avoid 0 in denominator

In [12]:
def get_new_state(state_array, param_matrix, interaction_matrix, state_var_indices, n_genes, n_state_vars, num_cells,
                                   EC50_matrix=None, global_params=None, t=1/60, ):
    """
    Pure 3D matrix operations - everything happens simultaneously across all genes and cells.
    
    Args:
        state_array: shape (n_genes, n_state_vars, num_cells)
        param_matrix: DataFrame with genes as rows, parameters as columns
        interaction_matrix: shape (n_genes, n_genes) - regulatory interactions
        state_var_indices: dict mapping state variable names to indices
        global_params: dict of global parameters
        t: time step
    
    Returns:
        np.ndarray: updated state_array
    """

    if global_params is None:
        global_params = {"splicing_rate": np.log(2)/(7/60), "max_effect": 16, "n": 2}
    
    assert state_array.shape == (n_genes, n_state_vars, num_cells)
    new_state = state_array.copy()
    
    # Short aliases
    s = state_var_indices
    
    # Global parameters
    splicing_rate = global_params.get("splicing_rate", np.log(2)/(7/60))
    max_effect = global_params.get("max_effect", 16)
    n = global_params.get("n", 2)
    
    # === STEP 1: DEGRADATION - ALL GENES & CELLS AT ONCE ===
    
    # Create degradation rate arrays from DataFrame columns
    mrna_deg_rates = param_matrix['mRNA_degradation_rate'].values.reshape(n_genes, 1, 1)
    protein_deg_rates = param_matrix['protein_degradation_rate'].values.reshape(n_genes, 1, 1)
    
    # Apply degradation to entire state array slices - checked the dimensions
    unspliced_degraded = np.random.poisson(
        mrna_deg_rates * new_state[:, s["unspliced_mRNA"]:s["unspliced_mRNA"]+1, :] * t
    )
    spliced_degraded = np.random.poisson(
        mrna_deg_rates * new_state[:, s["spliced_mRNA"]:s["spliced_mRNA"]+1, :] * t
    )
    protein_degraded = np.random.poisson(
        protein_deg_rates * new_state[:, s["protein"]:s["protein"]+1, :] * t
    )
    
    # Update entire slices at once
    new_state[:, s["unspliced_mRNA"], :] = np.maximum(
        new_state[:, s["unspliced_mRNA"], :] - unspliced_degraded.squeeze(1), 0
    )
    new_state[:, s["spliced_mRNA"], :] = np.maximum(
        new_state[:, s["spliced_mRNA"], :] - spliced_degraded.squeeze(1), 0
    )
    new_state[:, s["protein"], :] = np.maximum(
        new_state[:, s["protein"], :] - protein_degraded.squeeze(1), 0
    )
    
    # === STEP 2: TRANSCRIPTION - ALL GENES & CELLS AT ONCE ===
    
    # Transcription rates from DataFrame column
    txn_rates = param_matrix['transcription_rate'].values.reshape(n_genes, 1)
    
    # Generate new mRNA for ALL genes and cells simultaneously
    new_mRNA = txn_rates * new_state[:, s["is_bursting"], :] * t
    
    # Add to unspliced mRNA pool - entire matrix operation
    new_state[:, s["unspliced_mRNA"], :] += new_mRNA
    new_state[:, s["mRNA_ever_produced"], :] += new_mRNA
    
    # === STEP 3: SPLICING - ALL GENES & CELLS AT ONCE ===
    
    # Splicing: unspliced → spliced for entire matrix
    spliced_amount = splicing_rate * new_state[:, s["unspliced_mRNA"], :] * t
    
    new_state[:, s["unspliced_mRNA"], :] -= spliced_amount
    new_state[:, s["spliced_mRNA"], :] += spliced_amount
    
    # === STEP 4: TRANSLATION - ALL GENES & CELLS AT ONCE ===
    
    # Protein production rates from DataFrame column
    protein_prod_rates = param_matrix['protein_production_rate'].values.reshape(n_genes, 1)
    
    # Create protein for ALL genes and cells simultaneously
    new_protein = protein_prod_rates * new_state[:, s["spliced_mRNA"], :] * t
    
    new_state[:, s["protein"], :] += new_protein
    new_state[:, s["protein_ever_produced"], :] += new_protein
    
    # Update total mRNA - entire matrix operation
    new_state[:, s["total_mRNA"], :] = (new_state[:, s["unspliced_mRNA"], :] + 
                                        new_state[:, s["spliced_mRNA"], :])
    
    # === STEP 5: K_ON ADJUSTMENT ===
    

    # === Additive interaction === #
    if interaction_matrix is not None and np.any(interaction_matrix != 0):
        protein_levels = new_state[:, s["protein"], :]  # shape: (n_genes, num_cells)

        if EC50_matrix is not None and EC50_matrix.ndim == 1:
            # Use EC50 vector (per regulator gene)
            EC50_vector = EC50_matrix.reshape(n_genes, 1)  # shape: (n_genes, 1)
            hill_responses = (protein_levels ** n) / (EC50_vector ** n + protein_levels ** n)  # (n_genes, num_cells)

            # Apply regulatory weights (interaction matrix) to hill_responses
            regulatory_effects = interaction_matrix.T @ hill_responses  # shape: (n_genes, num_cells)
        else:
            raise ValueError("EC50 input must be a 1D array when using vectorized logic.")
        
        base_k_on = param_matrix['k_on'].values.reshape(n_genes, 1)
        k_on_adjusted = base_k_on + max_effect*regulatory_effects
    else:
        k_on_adjusted = np.tile(param_matrix['k_on'].values.reshape(n_genes, 1), (1, num_cells))
    k_on_adjusted = np.maximum(k_on_adjusted, 1e-10)  # Avoid zero in denominator
    
    # Store in state array
    new_state[:, s["k_on_adjusted"], :] = k_on_adjusted


    # # === Multiplicative interaction === #
    # if interaction_matrix is not None and np.any(interaction_matrix != 0):
    #     protein_levels = new_state[:, s["protein"], :]  # shape: (n_genes, num_cells)

    #     if EC50_matrix is not None and EC50_matrix.ndim == 1:
    #         EC50_vector = EC50_matrix.reshape(n_genes, 1)
    #         hill_responses = (protein_levels ** n) / (EC50_vector ** n + protein_levels ** n)

    #         # Clip to avoid log(0)
    #         hill_clipped = np.clip(hill_responses, 1e-6, 1)

    #         # log-sum-exp for product
    #         log_effects = interaction_matrix.T @ np.log(hill_clipped)
    #         product_effects = np.exp(log_effects)  # shape: (n_genes, num_cells)

    #         # Scale product by number of non-zero regulators per target gene
    #         regulator_counts = (interaction_matrix != 0).sum(axis=0).reshape(-1, 1)  # shape: (n_genes, 1)
    #         scaled_effects = regulator_counts * product_effects

    #     else:
    #         raise ValueError("EC50 must be a 1D array.")

    #     base_k_on = param_matrix['k_on'].values.reshape(n_genes, 1)
    #     k_on_adjusted = base_k_on + max_effect * scaled_effects
    # else:
    #     k_on_adjusted = np.tile(param_matrix['k_on'].values.reshape(n_genes, 1), (1, num_cells))

    # k_on_adjusted = np.maximum(k_on_adjusted, 1e-10)
    # new_state[:, s["k_on_adjusted"], :] = k_on_adjusted
    
    # === STEP 6: BURST SWITCHING - ALL GENES & CELLS AT ONCE ===
    
    # Current burst states
    is_bursting = new_state[:, s["is_bursting"], :].astype(bool)
    
    # k_off rates from DataFrame column
    k_off_rates = param_matrix['k_off'].values.reshape(n_genes, 1)
    
    # Switch probabilities for ALL genes and cells simultaneously
    switch_off_prob = np.random.exponential(1 / k_off_rates, size=(n_genes, num_cells)) < t
    switch_on_prob = np.random.exponential(1 / k_on_adjusted) < t
    
    # Apply switching logic - pure boolean operations on entire matrices
    should_switch_off = is_bursting & switch_off_prob
    should_switch_on = (~is_bursting) & switch_on_prob
    
    # Update burst states for entire matrix
    new_burst_state = (is_bursting | should_switch_on) & (~should_switch_off)
    new_state[:, s["is_bursting"], :] = new_burst_state.astype(float)
    
    # === FINAL: ENSURE NON-NEGATIVE VALUES ===
    new_state = np.maximum(new_state, 0)
    
    return new_state


In [13]:
def create_random_cells(n_genes, num_cells=1000, mean_protein_levels=None):
    """
    Create a random state array for a given number of genes and cells.
    
    Args:
        n_genes (int): Number of genes
        num_cells (int): Number of cells
    
    Returns:
        np.ndarray: Random state array of shape (n_genes, n_state_vars, num_cells)
    """
    state_vars_per_gene = ["is_bursting", "unspliced_mRNA", "spliced_mRNA", "protein",
                          "k_on_adjusted", "k_off_adjusted", "total_mRNA", 
                          "mRNA_ever_produced", "protein_ever_produced"]
    
    state_var_indices = {var: i for i, var in enumerate(state_vars_per_gene)}
    
    # Initialize state array with zeros
    state_array = np.zeros((n_genes, len(state_vars_per_gene), num_cells))
    #Protein levels for each gene are between 0 and 3*mean_protein_levels for the gene
    if mean_protein_levels is None:
        mean_protein_levels = np.random.uniform(0, 100, n_genes)
    else:
        mean_protein_levels = np.asarray(mean_protein_levels).reshape(n_genes)

    # Randomly initialize protein levels
    state_array[:, state_var_indices["protein"], :] = np.random.uniform(
        0, mean_protein_levels[:, np.newaxis], size=(n_genes, num_cells)
    )
    return state_array, state_var_indices

In [14]:
def select_parameter_rows(csv_path, rows=None, n_random=3):
    """
    Select parameter rows and return matrix format.
    
    Args:
        csv_path (str): Path to CSV file
        rows (list, optional): Specific row indices to select
        n_random (int): Number of random rows if rows=None
    
    Returns:
        tuple: (param_matrix, row_mapping)
    """
    df = pd.read_csv(csv_path, index_col=0)
    
    if rows is None:
        rows = np.random.choice(df.index, size=min(n_random, len(df)), replace=False)
    
    param_dict = {}
    row_mapping = {}
    
    for i, row in enumerate(rows):
        if row not in df.index:
            raise ValueError(f"Row {row} not found in the DataFrame.")
        gene_id = f"gene_{i+1}"
        param_dict[gene_id] = df.loc[row].to_dict()
        row_mapping[gene_id] = row
    
    # Create parameter matrix: genes as rows, parameters as columns
    param_matrix = pd.DataFrame.from_dict(param_dict, orient='index')
    
    return param_matrix, row_mapping


In [15]:
def simulate(path_to_matrix, parameter_sheet_path, num_cells, global_params= None, rows=None, threshold_frac = None):
    '''
    Simulates gene regulatory network dynamics over time for a given interaction matrix and parameter sheet.
    Args:
        path_to_matrix (str): Path to the input matrix file containing gene interaction data.
        parameter_sheet_path (str): Path to the parameter sheet file containing gene-specific parameters.
        num_cells (int): Number of cells to simulate before division.
        global_params (dict, optional): Dictionary of global parameters for the simulation. Defaults to:
            - "splicing_rate": Splicing rate in per minute.
            - "max_effect": List of max_effect for Hill function.
            - "n": Hill coefficient.
        rows (list, optional): List of rows to select from the parameter sheet. Defaults to None.
    Returns:
        pd.DataFrame: Combined DataFrame containing simulation results for both original and replicate states.
            - Columns include gene states, time steps, and replicate information.
    '''
    if global_params is None:
        # Default global parameters if not provided
        global_params = {
            "splicing_rate": np.log(2)/(7/60),  # Splicing rate in per minute
            "max_effect": 0.27,  # Maximum effect for Hill function
            "n": 2,  # Hill coefficient
        }
    print("Reading input matrix and parameters...")
    n_genes, interaction_matrix = read_input_matrix(path_to_matrix)
    print(interaction_matrix)
    param_matrix, row_mapping = assign_parameters_to_genes(parameter_sheet_path, rows=rows, n_random=n_genes)

    #Calculate EC50 matrix
    print("Calculating EC50 matrix...")
    print(f"Threshold fraction = {threshold_frac}")
    mean_protein_values, EC50_vector = create_EC50_vector(param_matrix, interaction_matrix, global_params,  threshold_frac = threshold_frac)
    print(EC50_vector)
    #Burn in - not needed because I already have mean and burn-in is just giving me one more mean

    #Create random cells
    print("Creating random cells...")
    state_array, state_var_indices = create_random_cells(n_genes, num_cells=num_cells, mean_protein_levels=mean_protein_values)
    print(state_var_indices)
    #simulation time settings
    t = 1/(60)  # 1 minute time step
    total_time = 13 * 24 * 60
    time_to_start_measurement = 12 * 24 * 60 # Start measuring after 12 days
    frequency_measurement = 60 # Measure every 60 minutes\
    steps = np.arange(0, total_time + 1, int(t*60))
    division_time = 12 * 24 * 60  # Division occurs after 12 days
    pre_division_steps = np.arange(0, min(total_time, division_time), 1)
    after_division_steps = steps[steps > division_time] - division_time
    state_list = []
    replicate_list = []
    print("Starting simulation...")
    for curr_step in tqdm(pre_division_steps, desc="Simulating before division"):
        state_array = get_new_state(state_array.copy(), param_matrix, interaction_matrix, state_var_indices, 
                                   n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
    print("Simulation before division complete.")
    replicate_state = state_array.copy()
    state_list.append(save_complete_snapshot(state_array, state_var_indices, timestep=0))
    replicate_list.append(save_complete_snapshot(replicate_state, state_var_indices, timestep=0))

    for curr_step in tqdm(after_division_steps, desc="Simulating after division"):
        state_array = get_new_state(state_array.copy(), param_matrix, interaction_matrix, state_var_indices, 
                                   n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
        replicate_state = get_new_state(replicate_state.copy(), param_matrix, interaction_matrix, state_var_indices, 
                                   n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
        
        if curr_step >= time_to_start_measurement - division_time and curr_step % frequency_measurement == 0:
            state_list.append(save_complete_snapshot(state_array, state_var_indices, timestep=curr_step))
            replicate_list.append(save_complete_snapshot(replicate_state, state_var_indices, timestep=curr_step))
    
    print("Simulation after division complete.")
    state_array_df = pd.concat(state_list, ignore_index=True)
    state_array_df['replicate'] = 0
    replicate_array_df = pd.concat(replicate_list, ignore_index=True)
    replicate_array_df['replicate'] = 1
    # Combine both dataframes
    combined_df = pd.concat([state_array_df, replicate_array_df], ignore_index=True)
    # Save the combined DataFrame to a CSV file

    return combined_df



In [16]:
# def simulate(path_to_matrix, parameter_sheet_path, num_cells, global_params= None, rows=None, threshold_frac = None):
#     '''
#     Simulates gene regulatory network dynamics over time for a given interaction matrix and parameter sheet.
#     Args:
#         path_to_matrix (str): Path to the input matrix file containing gene interaction data.
#         parameter_sheet_path (str): Path to the parameter sheet file containing gene-specific parameters.
#         num_cells (int): Number of cells to simulate before division.
#         global_params (dict, optional): Dictionary of global parameters for the simulation. Defaults to:
#             - "splicing_rate": Splicing rate in per minute.
#             - "max_effect": List of max_effect for Hill function.
#             - "n": Hill coefficient.
#         rows (list, optional): List of rows to select from the parameter sheet. Defaults to None.
#     Returns:
#         pd.DataFrame: Combined DataFrame containing simulation results for both original and replicate states.
#             - Columns include gene states, time steps, and replicate information.
#     '''
#     if global_params is None:
#         # Default global parameters if not provided
#         global_params = {
#             "splicing_rate": np.log(2)/(7/60),  # Splicing rate in per minute
#             "max_effect": 0.27,  # Maximum effect for Hill function
#             "n": 2,  # Hill coefficient
#         }
#     print("Reading input matrix and parameters...")
#     n_genes, interaction_matrix = read_input_matrix(path_to_matrix)
#     print(interaction_matrix)
#     param_matrix, row_mapping = assign_parameters_to_genes(parameter_sheet_path, rows=rows, n_random=n_genes)

#     #Calculate EC50 matrix
#     print("Calculating EC50 matrix...")
#     print(f"Threshold fraction = {threshold_frac}")
#     mean_protein_values, EC50_vector = create_EC50_vector(param_matrix, interaction_matrix, global_params,  threshold_frac = threshold_frac)
#     print(EC50_vector)
#     #Burn in - not needed because I already have mean and burn-in is just giving me one more mean

#     #Create random cells
#     print("Creating random cells...")
#     state_array, state_var_indices = create_random_cells(n_genes, num_cells=num_cells, mean_protein_levels=mean_protein_values)
#     print(state_var_indices)
#     #simulation time settings
#     t = 1/(60* 60)  # 1 minute time step
#     total_time = 13 * 24 * 60 * 60
#     time_to_start_measurement = 12 * 24 * 60 * 60 # Start measuring after 12 days
#     frequency_measurement = 60* 60  # Measure every 60 minutes\
#     steps = np.arange(0, total_time + 1, int(t*60*60))
#     division_time = 12 * 24 * 60* 60  # Division occurs after 12 days
#     pre_division_steps = np.arange(0, min(total_time, division_time), 1)
#     after_division_steps = steps[steps > division_time] - division_time
#     state_list = []
#     replicate_list = []
#     print("Starting simulation...")
#     for curr_step in tqdm(pre_division_steps, desc="Simulating before division"):
#         state_array = get_new_state(state_array.copy(), param_matrix, interaction_matrix, state_var_indices, 
#                                    n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
#     print("Simulation before division complete.")
#     replicate_state = state_array.copy()
#     state_list.append(save_complete_snapshot(state_array, state_var_indices, timestep=0))
#     replicate_list.append(save_complete_snapshot(replicate_state, state_var_indices, timestep=0))

#     for curr_step in tqdm(after_division_steps, desc="Simulating after division"):
#         state_array = get_new_state(state_array.copy(), param_matrix, interaction_matrix, state_var_indices, 
#                                    n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
#         replicate_state = get_new_state(replicate_state.copy(), param_matrix, interaction_matrix, state_var_indices, 
#                                    n_genes = n_genes, n_state_vars = len(state_var_indices), num_cells = num_cells, EC50_matrix = EC50_vector, global_params=global_params, t=t)
        
#         if curr_step >= time_to_start_measurement - division_time and curr_step % frequency_measurement == 0:
#             state_list.append(save_complete_snapshot(state_array, state_var_indices, timestep=curr_step))
#             replicate_list.append(save_complete_snapshot(replicate_state, state_var_indices, timestep=curr_step))
    
#     print("Simulation after division complete.")
#     state_array_df = pd.concat(state_list, ignore_index=True)
#     state_array_df['replicate'] = 0
#     replicate_array_df = pd.concat(replicate_list, ignore_index=True)
#     replicate_array_df['replicate'] = 1
#     # Combine both dataframes
#     combined_df = pd.concat([state_array_df, replicate_array_df], ignore_index=True)
#     # Save the combined DataFrame to a CSV file

#     return combined_df



In [17]:
# #Simulating repression by reducing k_on


# path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/matrix101.txt"
# path_to_param = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/parameter_sheet101.csv"
# num_cells = 10000
# rows = [0, 0]
# output_folder = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/r_add_k_on_repression/"
# os.makedirs(output_folder, exist_ok=True)
# r_add_range = np.arange(0.0, 2.01, 0.05)
# print(f"Total runs: {len(r_add_range)}")

# def run_simulation_for_r_add(r_add, n = 2):
#     global_params = {
#         "splicing_rate": np.log(2) / (7 / 60),
#         "n": n,
#         "max_effect": r_add,
#     }
    
#     try:
#         filename = f"simulation_matrix101_A-|B_mod_kOn_rAdd_{r_add:.2f}_n_{n}_min_k_on_B.csv"
#         file_path = os.path.join(output_folder, filename)
#         df = simulate(path_to_matrix, path_to_param, num_cells, global_params=global_params, rows=rows)
#         df.to_csv(file_path, index=False)
#         return f"✅ r_add={r_add:.2f} done"
#     except Exception as e:
#         return f"❌ r_add={r_add:.2f} failed: {e}"

# # Parallel execution with progress bar
# with tqdm_joblib(tqdm(desc="Running simulations", total=len(r_add_range))) as _:
#     results = Parallel(n_jobs=8)(
#         delayed(run_simulation_for_r_add)(r_add) for r_add in r_add_range
#     )

# # Print summary
# for res in results:
#     print(res)


In [18]:
# #Simulating repression by modifying threshold in hill function


# path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/matrix101.txt"
# path_to_param = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/test_data/parameter_sheet101.csv"
# num_cells = 10000
# rows = [0, 0]
# output_folder = "/home/mzo5929/Keerthana/grnInference/simulationData/general_simulation_data/threshold_mod/"
# os.makedirs(output_folder, exist_ok=True)
# threshold_range = np.arange(0.0, 0.5, 0.01)
# print(f"Total runs: {len(threshold_range)}")

# def run_simulation_for_threshold(threshold_frac = None, r_add = 10, n = 2):
#     global_params = {
#         "splicing_rate": np.log(2) / (7 / 60),
#         "n": n,
#         "max_effect": r_add,
#     }
    
#     try:
#         filename = f"simulation_matrix101_A-|B_mod_kOn_threshold_{threshold_frac:.2f}_n_{n}_median_r_add_10.csv"
#         file_path = os.path.join(output_folder, filename)
#         df = simulate(path_to_matrix, path_to_param, num_cells, global_params=global_params, rows=rows, threshold_frac = threshold_frac)
#         df.to_csv(file_path, index=False)
#         return f"✅ r_add={r_add:.2f} done"
#     except Exception as e:
#         return f"❌ r_add={r_add:.2f} failed: {e}"

# # Parallel execution with progress bar
# with tqdm_joblib(tqdm(desc="Running simulations", total=len(threshold_range))) as _:
#     results = Parallel(n_jobs=8)(
#         delayed(run_simulation_for_threshold)(threshold) for threshold in threshold_range
#     )

# # Print summary
# for res in results:
#     print(res)

In [19]:
#Simulating repression by reducing k_on
path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_code_data/test_data/matrix101.txt"
path_to_param = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_code_data/test_data/parameter_sheet101.csv"
num_cells = 10000
rows = [0,11]
output_folder = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_data/test/"
os.makedirs(output_folder, exist_ok=True)
threshold_range = np.arange(0.0, 0.5, 0.01)
print(f"Total runs: {len(threshold_range)}")

# n = 2
n = 2
r_add = 1
splicing_half_life = 7
global_params = {
        "splicing_rate": np.log(2) / (splicing_half_life / 60),
        "n": n,
        "max_effect": r_add,
    }
filename = f"simulation_matrix101_A_B_different_params_0_11_fine_time_step.csv"
file_path = os.path.join(output_folder, filename)
df = simulate(path_to_matrix, path_to_param, num_cells, global_params=global_params, rows=rows, threshold_frac = None)
# df.to_csv(file_path, index=False)


Total runs: 50
Reading input matrix and parameters...
[[0 1]
 [0 0]]
Calculating EC50 matrix...
Threshold fraction = None
[ 68.74872801 337.15810795]
Creating random cells...
{'is_bursting': 0, 'unspliced_mRNA': 1, 'spliced_mRNA': 2, 'protein': 3, 'k_on_adjusted': 4, 'k_off_adjusted': 5, 'total_mRNA': 6, 'mRNA_ever_produced': 7, 'protein_ever_produced': 8}
Starting simulation...


Simulating before division:   0%|          | 0/17280 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
#Simulating repression by reducing k_on
path_to_matrix = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_code_data/test_data/matrix101.txt"
path_to_param = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_code_data/test_data/parameter_sheet101.csv"
num_cells = 10000
rows = [12,13]
output_folder = "/home/mzo5929/Keerthana/grnInference/simulation_data/general_simulation_data/test/"
os.makedirs(output_folder, exist_ok=True)
threshold_range = np.arange(0.0, 0.5, 0.01)
print(f"Total runs: {len(threshold_range)}")

# n = 2
n = 4.4118197399108965
r_add = 1
splicing_half_life = 2.8486148303766226
global_params = {
        "splicing_rate": np.log(2) / (splicing_half_life / 60),
        "n": n,
        "max_effect": r_add,
    }
filename = f"simulation_matrix101_A_B_different_params_12_13_normal_time_step.csv"
file_path = os.path.join(output_folder, filename)
df = simulate(path_to_matrix, path_to_param, num_cells, global_params=global_params, rows=rows, threshold_frac = None)
df.to_csv(file_path, index=False)


Total runs: 50
Reading input matrix and parameters...
[[0 1]
 [0 0]]
Calculating EC50 matrix...
Threshold fraction = None
[  63.74968599 8210.63231119]
Creating random cells...
{'is_bursting': 0, 'unspliced_mRNA': 1, 'spliced_mRNA': 2, 'protein': 3, 'k_on_adjusted': 4, 'k_off_adjusted': 5, 'total_mRNA': 6, 'mRNA_ever_produced': 7, 'protein_ever_produced': 8}
Starting simulation...


Simulating before division:   0%|          | 0/17280 [00:00<?, ?it/s]

Simulation before division complete.


Simulating after division:   0%|          | 0/1440 [00:00<?, ?it/s]

Simulation after division complete.
